In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, CSVLogger

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPoolingV2, RandomLRF

import numpy as np
import sklearn as sk
import pickle

/home/vle/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
top_words=10000
sparse=False
remove_short_documents=True
notebook = 'mcNet_top10k_temptative_42'

In [4]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=top_words, sparse=sparse,
                remove_short_documents=remove_short_documents, verbose=False)

In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
num_classes = Y_train.shape[1]

In [6]:
%%time
#Process next cell only once
path = os.path.join('probabilities_' + 
                    'top' + str(top_words) +
                    '_sparse' + str(sparse) +
                    '_removeShorts' + str(remove_short_documents) +
                    '.pkl')
if os.path.isfile(path):
  probabilities = pickle.load(open(path, "rb"), encoding='latin1')

CPU times: user 2.29 s, sys: 739 ms, total: 3.03 s
Wall time: 3.03 s


In [7]:
if not(os.path.isfile(path)):
  METRIC = 'cosine'#'euclidean'
  distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

  # enforce exact zero
  for k in xrange(distances.shape[0]):
    distances[k,k] = 0.

  # max normalize
  #distances /= distances.max()
  distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

  # use tricube kernel (becaause of flatness around 0)
  probabilities = (1. - np.abs(distances) ** 3) ** 3

  # remove auto connections (which are taken anyway in LRF)
  for k in xrange(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # normalize proba
  probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))
  
  # pickled for later use
  pickle.dump(probabilities, open(path,"wb"))  

In [8]:
batch_size = 128
model = Sequential()
model.add(RandomLRF(probabilities, LRF_size=16, filters=25, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(RandomLRF(probabilities, LRF_size=16, filters=25, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(Conv1D(1, kernel_size=1, activation='relu',
#                 padding='same',kernel_initializer='he_uniform'))
#model.add(SeparableMonteCarloMaxPoolingV2(LRF_size=4, new_size=2500))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(500, activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_lrf_1 (RandomLRF)     (None, 10000, 25)         425       
_________________________________________________________________
random_lrf_2 (RandomLRF)     (None, 10000, 25)         10025     
_________________________________________________________________
flatten_1 (Flatten)          (None, 250000)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 250000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                5000020   
Total params: 5,010,470
Trainable params: 5,010,470
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
csv = CSVLogger(notebook + '_log.csv')

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1,
                    #callbacks=[early_stopper, csv],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/100
10168/10168 [==============================] - 27s 3ms/step - loss: 2.6814 - acc: 0.2787 - val_loss: 2.1871 - val_acc: 0.5506
Epoch 2/100
10168/10168 [==============================] - 25s 2ms/step - loss: 1.3499 - acc: 0.6961 - val_loss: 1.3709 - val_acc: 0.6439
Epoch 3/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.6877 - acc: 0.8159 - val_loss: 1.2831 - val_acc: 0.6669
Epoch 4/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.4617 - acc: 0.8704 - val_loss: 1.3264 - val_acc: 0.6713
Epoch 5/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.3491 - acc: 0.9002 - val_loss: 1.4042 - val_acc: 0.6672
Epoch 6/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.2728 - acc: 0.9207 - val_loss: 1.4897 - val_acc: 0.6661
Epoch 7/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.2264 - acc: 0.9345 - val_loss: 1.5806 - v

10168/10168 [==============================] - 25s 2ms/step - loss: 0.0588 - acc: 0.9829 - val_loss: 2.7412 - val_acc: 0.6538
Epoch 60/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.0586 - acc: 0.9821 - val_loss: 2.6950 - val_acc: 0.6518
Epoch 61/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.0521 - acc: 0.9847 - val_loss: 2.7406 - val_acc: 0.6529
Epoch 62/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.0446 - acc: 0.9859 - val_loss: 2.6532 - val_acc: 0.6497
Epoch 63/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.0506 - acc: 0.9845 - val_loss: 2.7107 - val_acc: 0.6497
Epoch 64/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.0583 - acc: 0.9832 - val_loss: 2.6303 - val_acc: 0.6517
Epoch 65/100
10168/10168 [==============================] - 25s 2ms/step - loss: 0.0514 - acc: 0.9852 - val_loss: 2.6883 - val_acc: 0.6508
Epoch 66/100
10168/10168 [==============

In [10]:
%%bash
tail mcNet_top10k_temptative_42_log.csv

epoch,acc,loss,val_acc,val_loss
0,0.5808418563836021,1.6425235706647487,0.6755762969043194,1.2167736569888654
1,0.8869984262951897,0.4604991487777674,0.6921227553864885,1.169997093844761
2,0.9468922109513842,0.22660820962264724,0.6907085280424465,1.215745747671018
3,0.9761998427373795,0.12425905712869204,0.6724649974487573,1.3969360224311516
